In [1]:
from daq import daqDevice
import daqh
import numpy as np
import time
import sys
import os
sys.path.append('C:\\cofe_analysis\\utils_meinhold')
sys.path.append('C:\cofe_analysis\\utils_zonca')
sys.path.append('C:\\cofe_analysis\\utils_zonca\\demod')
import cofe_util as cu
sys.path.append('C:\\Anaconda3\\envs\\py27\\Scripts')
import h5py
%pylab

import cofe_util as cu

from Tkinter import Tk
from tkFileDialog import askopenfilename

acqmode=daqh.DaamInfinitePost

ImportError: No module named cofe_util

In [62]:
def get_data(nchan=4,freq=10,nseconds=5,comment='None',alerts=[58,59,60,118,119,120,178,179,180,238,239,240,298,299,300]):
    """
    function to simply aquire nchan a/d channels at rate freq
    for nseconds seconds
    """
    
    #outdata=np.zeros([nchan,nscans],dtype=float)
    dev=daqDevice('DaqBoard3001USB')
    gains=[]
    flags=[]
    chans=[]
    if nchan > 8:
        uchan=nchan-8
        for i in range(8):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
        for i in range(uchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(256+i)   #HERE is the famous fix where DaqX refs upper level dif channels!
    elif nchan<9:      
        for i in range(nchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
    acqmode = daqh.DaamNShot
    dev.AdcSetAcq(acqmode, postTrigCount = nseconds*freq)
    dev.AdcSetScan(chans,gains,flags)
    dev.AdcSetFreq(freq)
    #use the driver buffer here user buffer was very limited (the way I tried anyway) 
    transMask = daqh.DatmUpdateBlock|daqh.DatmCycleOn|daqh.DatmDriverBuf

    buf=dev.AdcTransferSetBuffer(transMask, np.uint(nseconds*freq*nchan))
    #bufMask is for transferring the buffer
    bufMask = daqh.DabtmOldest | daqh.DabtmRetAvail

    timestart = (time.time())
    timenotify = timestart + 5

    dev.SetTriggerEvent(daqh.DatsImmediate,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStartEvent)
    dev.SetTriggerEvent(daqh.DatsScanCount,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStopEvent)
    dev.AdcTransferStart()
    dev.AdcArm()
    
    while True:
        alertscopy=alerts[:]
        timenotify = checkAlerts(timenotify, timestart, alerts,alertscopy)        
        
        stat = dev.AdcTransferGetStat()
        active = stat['active']
        if not (active & daqh.DaafAcqActive):
            break
    dev.AdcDisarm()
    outdata,ret=dev.AdcTransferBufData(nseconds*freq, nchan,bufMask)
    
    outdata=np.array(outdata,dtype=float)
    outdata=(outdata-2**15)*20./2**16
    outdata=np.transpose(np.reshape(outdata,[nseconds*freq,nchan]))
    print "Finished collecting data\n-"
    dev.Close()    
    return outdata

In [63]:
def import_data(filename=None):
    '''
    function to read in previously stored data and put in format of freshly read data dictionary
    '''
    
    if filename==None:
        Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        filename = askopenfilename(initialdir='C:\\Interferometer_tests\\io_3001_usb_data')
    hf=h5py.File(filename)
    d={}
    print(hf.keys())
    for key in hf.keys():
        d[key]=squeeze(array(hf[key]))
    hf.close()
    return d

In [4]:
def checkAlerts(timeupdate,timestart, alerts, alertscopy, updateincrement=5):
    timecheck = (time.time())
    if timecheck > timeupdate:
        print "Time: " + str(int(timecheck - timestart))
        timeupdate += updateincrement
    for alert in alerts:
        if (timecheck - timestart)%60 > alert:
            print "---- " + str(int(timecheck - timestart)) + " SECONDS ----"
            alerts.remove(alert)
    if 2<(timecheck -timestart%60)<3:
        alerts = alertscopy[:]
        
        
    return timeupdate

In [5]:
def get_date_filename(labelstring=''):
    #version for testing adds an optional string to identify tests
    now=time.localtime()[0:6]
    #dirfmt = "c:\\cofe\\ground_data\\testdata\\%4d_%02d_%02d"
    dirfmt = "C:\\Interferometer_tests\\io_3001_usb_data\\%4d_%02d_%02d"
    dirname = dirfmt % now[0:3]
    filefmt = "%02d_%02d_%02d.h5"
    filename= labelstring+filefmt % now[3:6]
    ffilename=os.path.join(dirname,filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    return(ffilename)

In [92]:
def zplot(d_dict_list,minfreq=1):
    """
    calculate asd's and overplot all from list of data dictionaries, assumed to have meaningful
    keynames for legend
    """
    figure()
    for d_dict in d_dict_list:
        z=cu.nps(d_dict['data'],d_dict['samprate'],minfreq=minfreq)
        plot(z[0],np.sqrt(z[1]),label=d_dict['label'])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    xscale('log')
    yscale('log')
    legend()
    title('Comparison spectra')
    legend()

In [96]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test(samprate=10000,nseconds=10,labelstring='test',minfreq=1):
    s=[0]  #active channels, intervening ones are terminated to increase isolation
    d=get_data(nchan=1,freq=samprate,nseconds=nseconds)
    #z1=cu.nps(d[0,:],samprate,minfreq)
    
    #figure()
    #plot(z1[0],np.sqrt(z1[1]),label='z1')
    #ylim([0,.05])
    #xlabel('Frequency, [Hz]')
    #label('ASD, [$V \sqrt{sec}$]')
    #title('Test')
    #legend()
    #yscale('log')
    #xscale('log')
    outdata=squeeze(d[s,:])
    fname=get_date_filename(labelstring)
    with h5py.File(fname, mode='w') as hdf_file:
        hdf_file.create_dataset('data', data=outdata)
        hdf_file.create_dataset('samprate', data=[samprate])
        hdf_file.create_dataset('label', data=[labelstring])
        hdf_file.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [97]:
t=get_test(nseconds=5,samprate=100000,labelstring='testing')

Time: 5
Finished collecting data
-


In [75]:
jnk.close()

In [76]:
jnk=h5py.File('jnk.hdf',mode='w')
jnk.create_dataset('data',data=a)

<HDF5 dataset "data": shape (10,), type "<i4">

In [83]:
jnk.create_dataset('label',data='testlabel')

<HDF5 dataset "label": shape (), type "|O">

In [84]:
jnk.close()

In [46]:
for key in hf.keys():
    d[key]=squeeze(array(hf[key]))
hf.close()

In [98]:
tf=import_data()

[u'data', u'label', u'samprate']


In [99]:
tf

{u'data': array([ 0.01220703,  0.01068115,  0.01159668, ...,  0.01159668,
         0.01220703,  0.01251221]), u'label': array('testing', 
       dtype='|S7'), u'samprate': array(100000)}

In [100]:
t

{'data': array([ 0.01220703,  0.01068115,  0.01159668, ...,  0.01159668,
         0.01220703,  0.01251221]), 'label': 'testing', 'samprate': 100000}

In [19]:
zplot([t1,t2,t3,t4,t5],minfreq=1)

In [48]:
t5=0
t6=0
t4=0
t3=0
t2=0
t1=0

In [23]:
td1=import_data()

[u'laser on,drop Tset']


In [105]:
zplot([tf,t])

In [21]:
td1.keys()

[u'laser on,drop Tset']

In [53]:
hf=h5py.File('tst.h5')

In [54]:
hf.create_dataset('a',a)

<HDF5 dataset "a": shape (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), type "<f4">

In [57]:
hf.create_dataset('label',['tstlabel'])

TypeError: Can't convert element 0 (tstlabel) to hsize_t